In [42]:
from gurobipy import *
import numpy as np
import math as mt

In [43]:
nb_var = 12
nb_cont = 7
n = 2
p = 12

In [44]:
# Matrice utilite
matrice_utilite = np.array (
                [
                        [-5, -10, -2, -4, -1, -4, -3, -1, -1, -3, -1, -1],
                        [-3, -4, -6, -2, -3, -6, -1, -2, -4, -5, -1, -1]
                ]
        )

print(matrice_utilite)


[[ -5 -10  -2  -4  -1  -4  -3  -1  -1  -3  -1  -1]
 [ -3  -4  -6  -2  -3  -6  -1  -2  -4  -5  -1  -1]]


In [45]:
# second membres
second_membre_1 = [0 for i in range(4)]

second_membre_existence_chemin =[0 for i in range(nb_cont)]
second_membre_existence_chemin[0] = 1
second_membre_existence_chemin[-1] = -1

second_membre = second_membre_1 + second_membre_existence_chemin

print(second_membre)

[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, -1]


In [46]:
# vecteur w
w = [2, 1]

# creer le vecteur w'
w_prime = [(w[k] - w[k+1]) for k in range(len(w)-1)]
w_prime.append(w[-1])
print(w_prime)

[1, 1]


In [47]:
## les variables rk
coef_rk = []
for k in range(n):
        for i in range(n):
                coef_rk.append([])
                for j in range(n):
                        coef_rk[-1].append(0)
                coef_rk[-1][k] = 1
coef_rk = np.array(coef_rk)
print(coef_rk)

[[1 0]
 [1 0]
 [0 1]
 [0 1]]


In [48]:
## les variables bik
coef_bik = -1 * np.identity(n**2)
print(coef_bik)

[[-1. -0. -0. -0.]
 [-0. -1. -0. -0.]
 [-0. -0. -1. -0.]
 [-0. -0. -0. -1.]]


In [53]:
## les variables x
coef_xi = []
for k in range(n**2):
        coef_xi.append([])
        for i in range(p):
                coef_xi[-1].append(-u[k%n][i])
coef_xi = np.array(coef_xi)
print(coef_xi)

[[ 5 10  2  4  1  4  3  1  1  3  1  1]
 [ 3  4  6  2  3  6  1  2  4  5  1  1]
 [ 5 10  2  4  1  4  3  1  1  3  1  1]
 [ 3  4  6  2  3  6  1  2  4  5  1  1]]


In [57]:
# contrainte de l'existence d'un chemin
existe_chemin = np.array([
                [1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
                [-1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
                [0, -1, 0, -1, 0, 0, 1, 1, -1, 0, 0, 0],
                [0, 0, -1, 0, -1, 0, 0, 0, 1, 1, 0, 0],
                [0, 0, 0, 0, 0, -1, -1, 0, 0, 0, 1, 0],
                [0, 0, 0, 0, 0, 0, 0, -1, 0, -1, 0, 1],
                [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1]
        ])

print(existe_chemin)

[[ 1  1  1  0  0  0  0  0  0  0  0  0]
 [-1  0  0  1  1  1  0  0  0  0  0  0]
 [ 0 -1  0 -1  0  0  1  1 -1  0  0  0]
 [ 0  0 -1  0 -1  0  0  0  1  1  0  0]
 [ 0  0  0  0  0 -1 -1  0  0  0  1  0]
 [ 0  0  0  0  0  0  0 -1  0 -1  0  1]
 [ 0  0  0  0  0  0  0  0  0  0 -1 -1]]


In [62]:
# concatenation des matrices rk, bik, xk 
result_concat = np.concatenate(
        (coef_rk, np.concatenate((coef_bik, coef_xi),axis=1)),
        axis=1)
print(result_concat)

[[ 1.  0. -1. -0. -0. -0.  5. 10.  2.  4.  1.  4.  3.  1.  1.  3.  1.  1.]
 [ 1.  0. -0. -1. -0. -0.  3.  4.  6.  2.  3.  6.  1.  2.  4.  5.  1.  1.]
 [ 0.  1. -0. -0. -1. -0.  5. 10.  2.  4.  1.  4.  3.  1.  1.  3.  1.  1.]
 [ 0.  1. -0. -0. -0. -1.  3.  4.  6.  2.  3.  6.  1.  2.  4.  5.  1.  1.]]


In [69]:

# ajout de la contrainte sur l'existence d'un chemin
new_lines = []
for i in range(nb_cont):
        new_lines.append([])
        for j in range(6):
                new_lines[-1].append(0)
new_lines = np.concatenate((new_lines, existe_chemin), axis=1)
constrainte = np.vstack([result_concat, new_lines])
print(constrainte)

[[ 1.  0. -1. -0. -0. -0.  5. 10.  2.  4.  1.  4.  3.  1.  1.  3.  1.  1.]
 [ 1.  0. -0. -1. -0. -0.  3.  4.  6.  2.  3.  6.  1.  2.  4.  5.  1.  1.]
 [ 0.  1. -0. -0. -1. -0.  5. 10.  2.  4.  1.  4.  3.  1.  1.  3.  1.  1.]
 [ 0.  1. -0. -0. -0. -1.  3.  4.  6.  2.  3.  6.  1.  2.  4.  5.  1.  1.]
 [ 0.  0.  0.  0.  0.  0.  1.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0. -1.  0.  0.  1.  1.  1.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0. -1.  0. -1.  0.  0.  1.  1. -1.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0. -1.  0. -1.  0.  0.  0.  1.  1.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1. -1.  0.  0.  0.  1.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1.  0. -1.  0.  1.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0. -1. -1.]]


In [71]:
## coefficient fonction objectif
c = [(i+1) * w_prime[i] for i in range(n)] # coef des r_k
# coef des b_ik
for i in range(n) :
    for j in range(n) :
        c.append(-1*w_prime[i])

c += [0 for i in range(p)] # ajout des coef de x
c = np.array(c)
print(c)

[ 1  2 -1 -1 -1 -1  0  0  0  0  0  0  0  0  0  0  0  0]


In [74]:
# Creer les models de la fonction objectif
m = Model("Question 4.2")

In [76]:
vars = ['ab', 'ac', 'ad', 'bc', 'bd', 'be', 'ce', 'cf', 'dc', 'df', 'eg', 'fg']
assert len(vars) == 12

x = []
for i in range(n):
            x.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="r%d" % (i+1)))
for i in range (n):
    for j in range(n):
        x.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="b%d%d" % ((j+1,(i+1)))))

for node in vars:
    x.append(m.addVar(vtype=GRB.CONTINUOUS, lb=0, name="x_%s" % node))
    
# maj du modele pour integrer les nouvelles variables
m.update()
print(x)

18
